# Packages

In [1]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ambrosia.designer import Designer

# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

# Draw some charts by cities and periods 

In [2]:
df_stats = read_bq("""
WITH live AS (SELECT event_dt_part,
                     t1.city_id,
                     t2.city_name,
                     t2.country_id,
                     t2.country_name,
                     COUNT(DISTINCT user_id) AS users_with_checks

              FROM indriver-e6e40.emart.swrve_event t1
                       JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                            ON
                                t1.city_id = t2.city_id
              WHERE 1 = 1
                AND name IN (
                  'client.verification_start.show'
                  )
                AND JSON_EXTRACT_SCALAR(payload
                        , '$.vertical') IN ('city', 'new_order')
                AND JSON_EXTRACT_SCALAR(payload
                        , '$.verification_flow') = 'liveness'
                AND event_dt_part >= '2024-11-25'
                AND t1.city_id IN
                    (4228, 4261, 4269, 4517, 833, 5568, 5575, 4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559,
                     4758, 4519)
              GROUP BY 1, 2, 3, 4, 5),
     rides AS (SELECT metric_date_utc,
                      country_id,
                      city_id,
                      SUM(orders_count) AS orders,
                      SUM(rides_count)  AS rides
               FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
               WHERE user_type = 'pass'
                 AND metric_date_utc >= '2024-11-25'
                 AND city_id IN
                     (4228, 4261, 4269, 4517, 833, 5568, 5575, 4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559,
                      4758, 4519)
               GROUP BY 1, 2, 3)
SELECT metric_date_utc,
       country_name,
       city_name,
       orders,
       rides,
       users_with_checks
FROM rides t1
         LEFT JOIN live t2
                   ON t1.city_id = t2.city_id
                       AND t1.metric_date_utc = t2.event_dt_part
""")

df_stats.head()

,metric_date_utc,country_name,city_name,orders,rides,users_with_checks
0,2024-12-09,None,None,13215,3398,<NA>
1,2024-11-26,Peru,Lima,634003,349547,4329
2,2024-11-26,Brazil,Natal,2839,974,161
3,2024-11-28,None,None,6214,2684,<NA>
4,2024-12-15,Mexico,Tijuana,34784,7488,2260


In [5]:
df_stats.groupby(['metric_date_utc', 'city_name'],as_index=False)['orders', 'rides', 'users_with_checks'].sum()

,metric_date_utc,city_name,orders,rides,users_with_checks
0,2024-11-25,Bogota,228978,91064,871
1,2024-11-25,Brasília,20329,8783,353
2,2024-11-25,Calama,16905,12852,329
3,2024-11-25,Campinas,7409,2916,110
4,2024-11-25,Concepción,4899,3039,275
...,...,...,...,...,...
303,2024-12-16,Natal,1064,126,136
304,2024-12-16,Puerto Montt,3207,1664,292
305,2024-12-16,Salvador,6751,1105,763
306,2024-12-16,Santiago,17238,5695,1239


In [19]:
columns = ['rides', 'orders', 'users_with_checks']

for column in columns:

    fig = px.line(df_stats.groupby(['metric_date_utc', 'city_name'],as_index=False)['orders', 'rides', 'users_with_checks'].sum()
                , x="metric_date_utc"
                , y=column
                , color='city_name'
                , title=f"<b>{column}</b>"
                , width=1400, height=600
                )

    fig.update_traces(textposition="top center")

    fig.update_layout(
                    xaxis_title="Dates"
                , yaxis_title="Incident rate"
    )


    fig.show()

In [18]:
fig = px.line(df_stats.groupby(['metric_date_utc', 'city_name'],as_index=False)['orders', 'rides', 'users_with_checks'].sum().query("city_name in ('Lima')")
            , x="metric_date_utc"
            , y=['rides', 'orders', 'users_with_checks']
            , title=f"<b>Lima</b> - Orders / Rides / Liveness checks"
            , width=1400, height=600
            )

fig.update_traces(textposition="top center")

fig.update_layout(
                xaxis_title="Dates"
            , yaxis_title="Incident rate"
)


fig.show()

In [2]:
df_reg = read_bq("""
SELECT t1.city_id,
       t2.city_name,
       DATE(t1.created) AS reg_date,
       COUNT(t1.id)     AS total_reg_users
FROM indriver-e6e40.ods_monolith.tbl_user t1
         JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
              ON
                  t1.city_id = t2.city_id
WHERE mode = 'client'
  AND DATE(t1.created) >= '2024-11-25'
  AND t1.city_id IN (4228, 4261, 4269, 4517, 833, 5568, 5575, 4143, 4199, 4200, 4197, 4153, 4530, 4755, 4538, 4559,
                     4758, 4519)
GROUP BY 1, 2, 3


""")

df_reg.head()

,city_id,city_name,reg_date,total_reg_users
0,4538,Campo Grande,2024-12-07,368
1,4538,Campo Grande,2024-12-06,297
2,4538,Campo Grande,2024-12-03,236
3,4538,Campo Grande,2024-12-01,229
4,4538,Campo Grande,2024-12-02,277


In [3]:
fig = px.line(df_reg.groupby(['reg_date', 'city_name'],as_index=False)['total_reg_users'].sum()
            , x="reg_date"
            , y=['total_reg_users']
            , color='city_name'
            , title=f"Total registered users by dates"
            , width=1400, height=600
            )

fig.update_traces(textposition="top center")

fig.update_layout(
                xaxis_title="Dates"
            , yaxis_title="Incident rate"
)


fig.show()